# Residual Analysis

## 1. ML Model

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from loguru import logger
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import matplotlib.pyplot as plt
from blog.model.regularisation import select_features, get_monotone_constraints
from blog.model.evaluation import show_model_results
from probatus.interpret import ShapModelInterpreter
from blog.data.lendingclub_dataset import LendingClubDataset
from blog.model.sampling import oot_split
from blog.model.regularisation import get_monotone_constraints
from blog.model.feature_transformations import get_simple_feature_transformation
from blog.model.optimise import getObjective, tune_model
from blog.model.evaluation import get_segment_rocauc
from blog.model.target_transformations import get_sample_weights

In [ ]:
dataset_name = "lendingclub"
target = "loan_status"
path = "../data/lc_accepted.csv"

initial_features = [
    "verification_status",
    "emp_title",
    "int_rate",
    "loan_amnt",
    "total_rec_int",
    "total_acc",
    "tot_cur_bal",
    "fico_range_low",
    "fico_range_high",
    "grade",
    "total_rev_hi_lim",
    "sub_grade",
    "initial_list_status",
    "purpose",
    "issue_d",
    "emp_length",
    "pub_rec_bankruptcies",
    "last_pymnt_amnt",
    "num_actv_bc_tl",
    "total_pymnt",
    "loan_status",
    "term",
    "home_ownership",
    "revol_util",
    "application_type",
    "addr_state",
    "inq_last_6mths",
    "pub_rec",
    "dti",
    "mort_acc",
    "revol_bal",
    "title",
    "annual_inc",
    "out_prncp",
    "open_acc",
    "installment",
    "home_ownership",
    "avg_cur_bal",
    "annual_inc",
    "num_tl_90g_dpd_24m"
]

# combine two lists without duplicates
initial_features = list(set(initial_features))
print(initial_features)
# ================== Read and clean the dataset.===========================================
logger.info(f"1.Read data")
lcd = LendingClubDataset()
X, y = lcd.get_data(path=path, use_cols=initial_features, dropna=False, sample=-1)
logger.info(f"\nTarget distribution:\n{y.value_counts(normalize=True)}")

# Split data into IT & OOT datasets.
data = X.copy()
data[target] = y

X_it, y_it, X_oot, y_oot = oot_split(
    df=data, split_date="Jun-2016", split_col="issue_d", target_col=target
)
# Original data dictionary
data_dict = {"xtrain": X_it, "ytrain": y_it, "xtest": X_oot, "ytest": y_oot}

# =================== Apply feature transformation =========================================
logger.info(f"2.Transform features")
#data_dict = get_simple_feature_transformation(data_dict_org)

# =================== Feature Selection =========================================
logger.info(f"3.Starting feature selection.")
# Select the best features based on SHAPRFE CV
# selected_features, fs_plot = select_features(data=data_dict, verbose=100)

# selected_features=['loan_amnt','int_rate','installment','grade','annual_inc','home_ownership',
#                    'emp_length','term','addr_state', 
#                    'fico_range_low', 'fico_range_high']

selected_features = [
    "emp_length",
    "addr_state",
    "revol_util",
    "revol_bal",
    "term",
    "num_actv_bc_tl",
    "total_acc",
    "application_type",
    "purpose",
    "grade",
    'dti',
    "home_ownership",
    "annual_inc",
    "num_tl_90g_dpd_24m"
]
logger.info(f"Final features :  {selected_features}")
# Update the data dictionary with the selected features.
data_dict["xtrain"] = data_dict["xtrain"][selected_features]
data_dict["xtest"] = data_dict["xtest"][selected_features]

model_params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "feature_pre_filter": False,
    "lambda_l1": 3.221919143923753e-08,
    "lambda_l2": 0.0016740960905730054,
    "num_leaves": 66,
    "feature_fraction": 0.6,
    "bagging_fraction": 1.0,
    "bagging_freq": 0,
    "min_child_samples": 20,
}

# model_params = tune_model(data_dict["xtrain"], data_dict["ytrain"], "rocauc")

# =================== Train Model =========================================
logger.info(f"5.Train Model ")
logger.info(f"Creating model with params : {model_params}")
mono_lgb = lgb.LGBMClassifier(**model_params)
sample_weight = get_sample_weights(data_dict)
mono_model = show_model_results(data=data_dict, model=mono_lgb,sample_weight=sample_weight)

## Error Analysis

In [ ]:
# Create a validation set, to do the error analysis.
X_train, X_val, y_train, y_val = train_test_split(data_dict["xtrain"], data_dict["ytrain"], test_size=0.2,stratify=data_dict["ytrain"])

In [ ]:
# Train the model
lgb_model = lgb.LGBMClassifier(**model_params)
sample_weight=get_sample_weights(data_dict={'xtrain':X_train,'ytrain':y_train})
lgb_model.fit(X_train,y_train,sample_weight=sample_weight)

yhat_name = 'p_loan_status'
preds1 = lgb_model.predict_proba(X_val)[:,1]

X_val_results = X_val.copy().reset_index(drop=True)
X_val_results[yhat_name] = preds1
X_val_results[target] = y_val.values


In [ ]:
# Calculate the log loss error.
from sklearn.metrics import log_loss
import numpy as np
resid_name = f'r_{target}' 
# calculate logloss residuals
X_val_results[resid_name] = -X_val_results[target]*np.log(X_val_results[yhat_name]) -(1 - X_val_results[target])*np.log(1 - X_val_results[yhat_name])   
# check that logloss is calculated correctly
print('Mean logloss residual: %.6f' % X_val_results[resid_name].mean())
print('Logloss from sklearn %.6f' % log_loss(y_val, preds1))

In [ ]:
# initialize figure
# Matplotlib figure inline
%matplotlib inline   
fig, ax_ = plt.subplots(figsize=(8, 8))         
valid_yhat_df = X_val_results.copy()
# plot groups with appropriate color
color_list = ['#000066', '#FF6200'] 
c_idx = 0
groups = valid_yhat_df.groupby(target) # define groups for levels of PAY_0
for name, group in groups:
    ax_.plot(group.p_loan_status, group.r_loan_status, 
             label=' '.join([target, str(name)]),
             marker='o', linestyle='', color=color_list[c_idx], alpha=0.3)
    c_idx += 1
    
# annotate plot
plt.xlabel(yhat_name)
plt.ylabel(resid_name)
ax_.legend(loc=1)
plt.title('Global Logloss Residuals')
plt.show()


Some high-magnitude outlying residuals are visible. 
* Who are these customers? 
* Why is the model so wrong about them? 
* And are they somehow exerting undue influence on other predictions? 

The model could be retrained without these individuals and retested as a potentially remediation strategy.

`loan_status = 1 Residuals`

In [ ]:
bins = [0, 30, 60, 100]
labels = ['low','medium','high']
valid_yhat_df['dti_binned'] = pd.cut(valid_yhat_df['dti'], bins=bins, labels=labels)
valid_yhat_df1 = valid_yhat_df[valid_yhat_df[target] == 1]
valid_yhat_df1 = valid_yhat_df1.sort_values(by=f'r_{target}', ascending=False).reset_index(drop=True)

In [ ]:
# use Seaborn FacetGrid for convenience
# some seaborn configs
import seaborn as sns
sns.set(font_scale=0.9)                                         # legible font size
sns.set_style('whitegrid', {'axes.grid': False})                # white background, no grid in plots
sns.set_palette(sns.color_palette(['#000066', '#FF6200']))            # consistent colors

# facet grid of residuals by PAY_0 
sorted_ = valid_yhat_df.sort_values(by='grade')                 # sort for better layout of by-groups
g = sns.FacetGrid(sorted_, col='grade', hue=target, col_wrap=4) # init grid
_ = g.map(plt.scatter, yhat_name, resid_name, alpha=0.4)        # plot points
_ = g.add_legend(bbox_to_anchor=(0.82, 0.2))                    # legend

In [ ]:
# facet grid of residuals by PAY_0 
sorted_ = valid_yhat_df.sort_values(by='dti_binned')                 # sort for better layout of by-groups
g = sns.FacetGrid(sorted_, col='dti_binned', hue=target, col_wrap=4) # init grid
_ = g.map(plt.scatter, yhat_name, resid_name, alpha=0.4)        # plot points
_ = g.add_legend(bbox_to_anchor=(0.82, 0.2))                    # legend

### Residual Decision Tree

A simple decision tree can be built, to highlight the failure modes for the model.
We can futher simplfy it by choosing only the highly important features and creating a high cutoff for the residuals.

In [ ]:
top5_feats = ['grade','dti','term','home_ownership','num_actv_bc_tl']
# Drop missing values
valid_yhat_df1 = valid_yhat_df1.dropna()
X_err = valid_yhat_df1[top5_feats]
y_err = valid_yhat_df1['r_loan_status'].apply(lambda x: 1 if x >= 1.5 else 0)
print(X_err.shape)

In [ ]:
valid_yhat_df1['r_loan_status'].hist()

In [ ]:

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.model_selection import StratifiedKFold

X_ohe = pd.get_dummies(X_err)
clf =  DecisionTreeClassifier(max_depth=2,random_state=0)
mean_auc = cross_val_score(clf, X_ohe, y_err, 
                           cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0),
                           scoring='roc_auc').mean()
print(f"AUC on residuals : {mean_auc}")
clf.fit(X_ohe,y_err)

In [ ]:
from dtreeviz.trees import dtreeviz # remember to load the package
viz = dtreeviz(clf, X_ohe, y,
                target_name="target",
                feature_names=X_ohe.columns,
                class_names=["low residual", "high residual"],)

viz